In [42]:

import requests 
import bs4 
import pandas 

import re 
import urllib.parse 
import io 
import json 
import os.path 
import os 
import csv

In [43]:
def getTextFromTribunePage(targetURL, file_1, file_2):
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    lst_1 = []
    lst_2 = []
    contentPTags = soup.findAll('p')
    
   
    if 'class-f' in targetURL:
         for ptag in contentPTags:
          
            new_s = re.sub(r'[\n,".—-]','',ptag.text) 
            new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                
            lst_2.append(re.sub(r'\(\d+\)|\s*$', '', new_s1))
            with open(file_1, 'a') as f:
                writer = csv.writer(f)
                writer.writerow([re.sub(r'\(\d+\)|\s*$', '', new_s1)] )
    if 'class-m' in targetURL:
        for ptag in contentPTags:
            new_s = re.sub(r'[\n,".—-]','',ptag.text) 
            new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                
          
            lst_1.append(re.sub(r'\(\d+\)|\s*$', '', new_s1))
            with open(file_2, 'a') as f:
                writer = csv.writer(f)
                writer.writerow([re.sub(r'\(\d+\)|\s*$', '', new_s1)] )
        
                
    return pandas.DataFrame({'Grooms-wanted' : pandas.Series(lst_1), 'Brides-wanted':pandas.Series(lst_2)})

In [44]:
def get_final_urls(year, base_url):


    url_request = requests.get(base_url)
    web_page_soup = bs4.BeautifulSoup(url_request.text,'html.parser')

    tagLinks = web_page_soup.findAll('a', attrs={'href': re.compile(year)})

    final_urls = set()

    for aTag in tagLinks:
            relurl = aTag.get('href')

            level_1_request = requests.get(relurl)
            level_1_soup = bs4.BeautifulSoup(level_1_request.text, 'html.parser')
            level_1_tagLinks = level_1_soup.findAll('a',href = re.compile('class'), class_=False)
           

            for a1Tag in level_1_tagLinks:
                relurl = a1Tag.get('href')
            
                level_2_url = 'https://www.tribuneindia.com/'+year+re.sub(r'^..','',relurl)
               
                final_urls.add(level_2_url)
    return final_urls
         

In [45]:

def url_csv(year, base_url):
    final_df = pandas.DataFrame()
    final_urls = get_final_urls(year, base_url)
    for url in final_urls:
        
        try:
            print(url)
            final_df = final_df.append(getTextFromTribunePage(url,'grooms-wanted_'+year+'.csv','brides-wanted_'+year+'.csv'),ignore_index = True)
        except:
            continue

In [50]:
url_csv('2010', 'https://www.tribuneindia.com/2014/forms/arc10.htm')

<ipython-input-43-137ccf9676c1>:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pandas.DataFrame({'Grooms-wanted' : pandas.Series(lst_1), 'Brides-wanted':pandas.Series(lst_2)})
